# Tutorial

The first step is to install stratopy. To install it, on a terminal run

```console
$ pip install stratopy
```

After installing, the first thing to do is to load the stratopy package. 

# The stratopy package

We always recommend to read the documentation.

In [1]:
import stratopy

print(stratopy.__doc__)

A Python package designed to easily manipulate CloudSat and GOES-R
and generate labeled images containing cloud types.

It consists in five modules:

- stratopy.cloudsat module
- stratopy.core module
- stratopy.goes module
- stratopy.IO module
- stratopy.merger module
 


This tutorial will guide you through all five modules.

## Using the IO Module

Stratopy is intended to be used by data scientists that are interested in trainning machine learning algorithms based on satellite imagery data. `Stratopy` grants the possibility of downloading images from remote servers via the IO module. 

In case of having access to CloudSat's remote server, then by defining your username and password as strings and by passing the route to the desired image it is possible to download it (insert your username and password): 

In [2]:
from stratopy import IO

username = "your.username"
password = "your.password"
route = "2B-CLDCLASS.P1_R05/2019/003/2019003151948_67564_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf"

IO.fetch_cloudsat(
    dirname=route,
    user=username,
    passwd=password,
    host="ftp.cloudsat.cira.colostate.edu",
    tag="stratopy-cloudsat",
)

,read_time,Longitude,Latitude,layer_0,layer_1,layer_2,layer_3,layer_4,layer_5,layer_6,layer_7,layer_8,layer_9
0,2019-01-03 15:19:58.450000000,156.896011,-0.006094,0,0,0,0,0,0,0,0,0,0
1,2019-01-03 15:19:58.609999967,156.893967,-0.015801,0,0,0,0,0,0,0,0,0,0
2,2019-01-03 15:19:58.769999933,156.891907,-0.025507,0,0,0,0,0,0,0,0,0,0
3,2019-01-03 15:19:58.929999900,156.889862,-0.035213,0,0,0,0,0,0,0,0,0,0
4,2019-01-03 15:19:59.089999986,156.887802,-0.044920,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36945,2019-01-03 16:58:29.651171875,132.272720,0.043348,0,0,0,0,0,0,0,0,0,0
36946,2019-01-03 16:58:29.810839844,132.270676,0.033642,0,0,0,0,0,0,0,0,0,0
36947,2019-01-03 16:58:29.970996094,132.268616,0.023935,0,0,0,0,0,0,0,0,0,0
36948,2019-01-03 16:58:30.131152344,132.266571,0.014229,0,0,0,0,0,0,0,0,0,0


## Loading data

If data is already present in your local computer, there is no need to use the IO module. The `cloudsat` and `goes` modules assists the user in loading the satellite image.

### About the Cloudsat Module

This module may be used to load a Cloudsat image. For this tutorial, there is a data folder in the stratopy data directory that will be used. To load a cloudsat image just import `stratopy.cloudsat`:

In [3]:
from stratopy import cloudsat as cs

print(cs.__doc__)

Module containing all CloudSat related classes and methods.


Then, using the `cs.read_hdf` method, one can simply import the hdf data as follows:

In [4]:
cloudsat_im_path = '../../data/CloudSat/2019002175851_67551_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf'
cloudsat_obj = cs.read_hdf(cloudsat_im_path)

It is posible to crop the image to a desired rectangle with defined latitude and longitude coordinates using the `cut` method:

In [5]:
cloudsat_obj = cloudsat_obj.cut([-40.,10., -78.,-36])

### About the Goes Module

If GOES satellite data is present in the local machine, one may import the goes module from stratopy:

In [6]:
from stratopy import goes

print(goes.__doc__)

Module containing all GOES satellite related classes and methods.


This package is only meant to process CMI (Cloud and Moisture) Full Disk products (Level 2) from GOES16. 
That said, in order to load a file you can simply use the `goes.read_nc` method. Here you have three options:
- You can open a single-channel file.
- You can open three pre-defined channels all-togheter: C03, CH7 and CH13. 
- You can open a multi-band CMI file, which contains all 16 abi bands processed as a CMI product.

#### Single-channel file

In [8]:
goes_path_1 = ("../../data/GOES16/OR_ABI-L2-CMIPF-M3C07_G16_s20190021800363_e20190021811141_c20190021811202.nc",)
goes_obj_1 = goes.read_nc(goes_path)

#### Three channels: the Day Microphysics product

If you'd like to open 3 bands (in order to generate the day microphysics RGB later), we just add the three paths to the path tuple and use the read_nc function:

In [ ]:
goes_path_3 = ("../../data/GOES16/OR_ABI-L2-CMIPF-M3C03_G16_s20190021800363_e20190021811129_c20190021811205.nc", #channel 3
                "../../data/GOES16/OR_ABI-L2-CMIPF-M3C07_G16_s20190021800363_e20190021811141_c20190021811202.nc", #channel 7
                "../../data/GOES16/OR_ABI-L2-CMIPF-M3C13_G16_s20190021800363_e20190021811141_c20190021811221.nc") #channel 13
goes_obj_3 = read_nc(goes_path_3)

Now, using goes_obj_3 it's possible to easily generate a *Day Microphysics RGB* product, where the Reds channel is composed by band 3, the Greens channel is composed by band 7 and the Blues channel is composed by band 13, by simply using the RGB method. This will return a numpy array of shape (a,l,3) where ...The default shape is (2441, 1666, 3) and includes a great portion of Southamerica. 

In [ ]:
rgb_img = rgb_data.RGB

#Plot the image 
plt.imshow(rgb_img)

#### Multi-band file

In GOES images it is also possible to trim the image to a specified rectangle:

You can always change the coordinates where you are trimming the Full-Disk image by....

In [9]:
# Return a dictionary
goes_cut = goes_obj.trim()

# Return the image array 
goes_array = goes_cut.get('M3C03')

## Using the merger Module

In [10]:
from stratopy import merger

print(merger.__doc__)

Contains the principal Data Frame class which allows for merging
satellite images.


In order to merge data from GOES16 and Cloudsat, we must have preaviously created a *goes object* and a *cloudsat object*. 

In [ ]:
merged_df = stratopy.merger.merge(cldsat_obj, goes_obj)

As we can see, this returns a .... object. This object inherits all the functionalities of a Pandas Dataframe object. 

In [ ]:
(merged_df), merged_df

The goes_vect colum is the column that contains a vector of shape *3x3xN* where N could be 
- 1 if the goes object corresponds to a one-band file (CMI)
- 3 if the goes object corresponds to a 3-band file (RGB)(CMI)
- 16 if the goes object corresponds to a 16-band file (MCMIPF)

In [ ]:
merged_df['goes_vec'].iloc[0].shape

## Understanding the core Module

In [11]:
from stratopy import core

print(core.__doc__)

Contains methods to perform transformation operations on loaded images.
